In [98]:
import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn import tree
#from sklearn.preprocessing import OneHotEncoder #get dummy variables for categories
#from sklearn.preprocessing import LabelEncoder #get dummy variables for categories  (the pandas get dummies is better)
from imblearn.over_sampling import SMOTE 
from imblearn.under_sampling import NearMiss 


In [99]:
# graphviz and pydotplus are used for decision tree visualization, need to install graphviz to both windows system and python need to add path to windows environments, refer to this link  https://stackoverflow.com/questions/18438997/why-is-pydot-unable-to-find-graphvizs-executables-in-windows-8
from IPython.display import Image 
import graphviz 
import pydotplus
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'


# 1. Prepare and clean data

In [100]:
os.chdir(r'D:\NDSUGoogle\Pyntebook\nlp analysis\LasVegasResult')
data=pd.read_csv("complete_data.csv")
data.head()

,Unnamed: 0,season,year,month,rate,0,1,2,3,4,...,20,21,22,23,24,25,26,27,28,29
0,5748,sp,17,Mar,C,0.002381,0.573810,0.002381,0.002381,0.002381,...,0.002381,0.002381,0.002381,0.002381,0.002381,0.359524,0.002381,0.002381,0.002381,0.002381
1,7853,a,16,Nov,D,0.093939,0.366667,0.003030,0.003030,0.093939,...,0.003030,0.003030,0.003030,0.003030,0.003030,0.366667,0.003030,0.003030,0.003030,0.003030
2,20335,sp,12,Mar,B,0.003704,0.337037,0.003704,0.003704,0.225926,...,0.003704,0.003704,0.003704,0.003704,0.003704,0.337037,0.003704,0.003704,0.003704,0.003704
3,906,sp,18,Apr,A,0.001449,0.305797,0.001449,0.001449,0.001449,...,0.001449,0.001449,0.001449,0.001449,0.001449,0.088900,0.001449,0.001449,0.001449,0.001449
4,20401,w,11,Dec,A,0.003030,0.275758,0.003030,0.003030,0.003030,...,0.003030,0.003030,0.003030,0.003030,0.003030,0.639394,0.003030,0.003030,0.003030,0.003030


In [101]:
dummies=data['season'].str.get_dummies()
dummies.head()

,a,sm,sp,w
0,0,0,1,0
1,1,0,0,0
2,0,0,1,0
3,0,0,1,0
4,0,0,0,1


In [102]:
seasondata=pd.concat([data,dummies],axis=1).drop(['Unnamed: 0'],axis=1)
seasondata.head()

,season,year,month,rate,0,1,2,3,4,5,...,24,25,26,27,28,29,a,sm,sp,w
0,sp,17,Mar,C,0.002381,0.573810,0.002381,0.002381,0.002381,0.002381,...,0.002381,0.359524,0.002381,0.002381,0.002381,0.002381,0,0,1,0
1,a,16,Nov,D,0.093939,0.366667,0.003030,0.003030,0.093939,0.003030,...,0.003030,0.366667,0.003030,0.003030,0.003030,0.003030,1,0,0,0
2,sp,12,Mar,B,0.003704,0.337037,0.003704,0.003704,0.225926,0.003704,...,0.003704,0.337037,0.003704,0.003704,0.003704,0.003704,0,0,1,0
3,sp,18,Apr,A,0.001449,0.305797,0.001449,0.001449,0.001449,0.001449,...,0.001449,0.088900,0.001449,0.001449,0.001449,0.001449,0,0,1,0
4,w,11,Dec,A,0.003030,0.275758,0.003030,0.003030,0.003030,0.003030,...,0.003030,0.639394,0.003030,0.003030,0.003030,0.003030,0,0,0,1


In [103]:
newdata=data.drop(['season','Unnamed: 0','year','month'],axis=1)
newdata.head()

,rate,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,C,0.002381,0.573810,0.002381,0.002381,0.002381,0.002381,0.002381,0.002381,0.002381,...,0.002381,0.002381,0.002381,0.002381,0.002381,0.359524,0.002381,0.002381,0.002381,0.002381
1,D,0.093939,0.366667,0.003030,0.003030,0.093939,0.003030,0.003030,0.003030,0.003030,...,0.003030,0.003030,0.003030,0.003030,0.003030,0.366667,0.003030,0.003030,0.003030,0.003030
2,B,0.003704,0.337037,0.003704,0.003704,0.225926,0.003704,0.003704,0.003704,0.003704,...,0.003704,0.003704,0.003704,0.003704,0.003704,0.337037,0.003704,0.003704,0.003704,0.003704
3,A,0.001449,0.305797,0.001449,0.001449,0.001449,0.001449,0.001449,0.001449,0.001449,...,0.001449,0.001449,0.001449,0.001449,0.001449,0.088900,0.001449,0.001449,0.001449,0.001449
4,A,0.003030,0.275758,0.003030,0.003030,0.003030,0.003030,0.003030,0.003030,0.003030,...,0.003030,0.003030,0.003030,0.003030,0.003030,0.639394,0.003030,0.003030,0.003030,0.003030


## change 12345 to pos,neg

In [106]:
## recode rate to 3 categories
newdata.loc[data['rate']=="A", 'rate2']=1
newdata.loc[data['rate']=="B", 'rate2']=1
newdata.loc[data['rate']=="C", 'rate2']=0
newdata.loc[data['rate']=="D", 'rate2']=0
newdata.loc[data['rate']=="E", 'rate2']=0
data2=newdata.drop(['rate'],axis=1)
data2.head()
## 五个精度低，特别少。。。  我们在意的是positive或者negtive。。而不是very positive或者very negtive。。

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,rate2
0,0.002381,0.573810,0.002381,0.002381,0.002381,0.002381,0.002381,0.002381,0.002381,0.002381,...,0.002381,0.002381,0.002381,0.002381,0.359524,0.002381,0.002381,0.002381,0.002381,0.0
1,0.093939,0.366667,0.003030,0.003030,0.093939,0.003030,0.003030,0.003030,0.003030,0.003030,...,0.003030,0.003030,0.003030,0.003030,0.366667,0.003030,0.003030,0.003030,0.003030,0.0
2,0.003704,0.337037,0.003704,0.003704,0.225926,0.003704,0.003704,0.003704,0.003704,0.003704,...,0.003704,0.003704,0.003704,0.003704,0.337037,0.003704,0.003704,0.003704,0.003704,1.0
3,0.001449,0.305797,0.001449,0.001449,0.001449,0.001449,0.001449,0.001449,0.001449,0.001449,...,0.001449,0.001449,0.001449,0.001449,0.088900,0.001449,0.001449,0.001449,0.001449,1.0
4,0.003030,0.275758,0.003030,0.003030,0.003030,0.003030,0.003030,0.003030,0.003030,0.003030,...,0.003030,0.003030,0.003030,0.003030,0.639394,0.003030,0.003030,0.003030,0.003030,1.0


In [107]:
## sum the weight for pos or neg, to estimate the direction of each topic
topicDirection=data2.groupby(['rate2']).mean()
topicDirection.to_csv('topicDirection.csv')

# 2. try random forest to explore data pre-processing

In [108]:
label.describe()

count     20473
unique        2
top         pos
freq      17315
Name: rate2, dtype: object

### set value <0.01 to 0 and use smote. OA decrease, but recall increase further

In [118]:
### smote通过增加 线性组合的sample把neg和postive的样本量变得差不多。。
##因为我们的数据大部分都是positive的。。
features=data2.drop(['rate2'],axis=1)
#features[features<0.02]=0
label=data2.rate2
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.05, random_state=333)
sm = SMOTE()
nm=NearMiss() #near missing is undersampling, which will reduce the number of training set
X_res, y_res = sm.fit_sample(X_train, y_train)

clf = RandomForestClassifier(random_state=42)
clf_fit=clf.fit(X_res, y_res)
y_predict=clf_fit.predict(X_test)
#evaluate y_predict performance
from sklearn.metrics import classification_report, confusion_matrix
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, y_predict))
print('\n')


=== Confusion Matrix ===
[[ 36  99]
 [ 55 834]]


=== Classification Report ===
              precision    recall  f1-score   support

         0.0       0.40      0.27      0.32       135
         1.0       0.89      0.94      0.92       889

    accuracy                           0.85      1024
   macro avg       0.64      0.60      0.62      1024
weighted avg       0.83      0.85      0.84      1024





In [119]:
importance=clf.feature_importances_
print(importance)
## this is the most important feature name
##用这个看重要不重要
largest5=importance.argsort()[-5:]
[list(features)[i] for i in largest5]


[0.02195538 0.02076753 0.02520255 0.018688   0.04486941 0.02220212
 0.02608338 0.02200368 0.02108351 0.0200318  0.0253033  0.03793029
 0.09652744 0.03029267 0.04490259 0.02084376 0.02310686 0.02283947
 0.03294735 0.02228456 0.02341881 0.02013446 0.02114298 0.06590224
 0.02040148 0.07944753 0.02277688 0.07237289 0.02091537 0.05362171]


['29', '23', '27', '25', '12']

### try use logistic model

In [122]:
from sklearn.linear_model import LogisticRegression

features=data2.drop(['rate2'],axis=1) 
features[features<0.1]=0
label=data2.rate2
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.05, random_state=4)
logmodel=LogisticRegression( class_weight='balanced')# the balanced option will assign inversed weights 
log_fit=logmodel.fit(X_train, y_train)
y_predict=log_fit.predict(X_test)

#evluate the model
from sklearn.metrics import classification_report
print (classification_report(y_test,y_predict))

#print coefficient
coef=logmodel.coef_
print (coef)


              precision    recall  f1-score   support

         0.0       0.30      0.69      0.41       161
         1.0       0.92      0.69      0.79       863

    accuracy                           0.69      1024
   macro avg       0.61      0.69      0.60      1024
weighted avg       0.82      0.69      0.73      1024

[[ 0.99603859  0.97997857  0.32109415  1.78474022  1.29591091 -0.46294697
  -0.34578906 -0.21774214 -0.10085974  0.86776886 -0.85052145  0.15558913
   1.35071232  0.34529082 -1.02268632  0.74755528  0.90426996  2.30212577
   0.4528454   0.85735631  0.51380997 -0.07094892 -0.47728944 -1.04013612
   2.56157378  4.94790164  2.23684212 -2.09589535  0.44532094  3.84008478]]


In [123]:
print (list(coef[0]))
from sklearn.metrics import r2_score
r2_score(y_test,y_predict)#the result is negative, meaning rsquare doesnot fit for classification problems.

[0.9960385897362978, 0.9799785685122737, 0.3210941533912557, 1.7847402165371933, 1.2959109128427928, -0.46294696666005325, -0.3457890565022754, -0.2177421407496199, -0.10085974450332809, 0.8677688609411798, -0.8505214524038588, 0.15558912933759914, 1.350712319416233, 0.3452908204517974, -1.0226863227311669, 0.7475552764708149, 0.9042699550604921, 2.302125769109948, 0.4528453963124849, 0.8573563083076265, 0.5138099734852941, -0.07094892301320305, -0.47728943847950833, -1.0401361205380535, 2.561573775347796, 4.947901643580021, 2.2368421230283246, -2.095895345153303, 0.44532094384309723, 3.8400847783049126]


-1.3141576041974048

# 3. Implement decision tree following C.1 for interpretation purpose

In [20]:
features=data2.drop(['rate2'],axis=1)
features[features<0.01]=0
label=data2.rate2
features.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.000000,0.573810,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.359524,0.0,0.0,0.0,0.0
1,0.093939,0.366667,0.0,0.0,0.093939,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.366667,0.0,0.0,0.0,0.0
2,0.000000,0.337037,0.0,0.0,0.225926,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.337037,0.0,0.0,0.0,0.0
3,0.000000,0.305797,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.088900,0.0,0.0,0.0,0.0
4,0.000000,0.275758,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.639394,0.0,0.0,0.0,0.0


In [21]:
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=42)
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_sample(X_train, y_train)

clf=tree.DecisionTreeClassifier(max_depth=4)  ## this max_depth is after trials among [3,4,5]
clf=clf.fit(X_res, y_res)
y_predict=clf.predict(X_test)
print(accuracy_score(y_test, y_predict))
print(recall_score(y_test, y_predict, average=None))
print(precision_score(y_test, y_predict, average=None))

0.6903540903540903
[0.53846154 0.71632828]
[0.24505327 0.90075512]


In [22]:
with open("decisionTree_classifier.txt", "w") as f:
    f = tree.export_graphviz(clf, out_file=f,feature_names=list(features),class_names=["pos",'neg'])
#Use http://webgraphviz.com/ to see the tree graphic

In [23]:
print (list(features))

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29']


In [ ]:
"""the result gini impurity look at how well the group been splitted. If all the group are Pos, then GINI is 0. If the pos and neg 
is 50%,50%, then the gini is 0.5, which is not good.48.97% means the 48.97% chance of a new data point being incorrectly 
classified, based on the observed training data we have at our disposal. 